# Générateur d'attestation de sortie automatique - COVID 19
*Ceci est une expérimentation, et n'est pas encore terminé*

## But initial

1. utiliser [Selenium](https://www.selenium.dev/) et [selenium Python](https://selenium-python.readthedocs.io/installation.html) pour automatiser l'utilisation du site [media.interieur.gouv.fr/deplacement-covid-19/](https://media.interieur.gouv.fr/deplacement-covid-19/),                                                                                                                             
2. sauvegarder le PDF localement, et l'envoyer par mail ou sur mon site web.

TODO ajouter warning etc.

- Auteur : [Lilian Besson](https://perso.crans.org/besson/) ([@Naereen](https://GitHub.com/Naereen) sur GitHub)
- Licence : [MIT Licensed](https://lbesson.mit-license.org/)
- Date : Samedi 31 octobre 2020

---
## Premier tutoriel pour prendre en main selenium

Note : je n'avais jamais utilisé selenium ou ce genre de module pour contrôler un navigateur en mode "headless".
Ce n'est pas très compliqué, et j'ai trouvé que ce petit tutoriel est bien fait : https://pythonbasics.org/selenium-firefox-headless/
[Cette documentation est bien complète, pour le module Python de selenium](https://selenium-python.readthedocs.io/).

In [66]:
from selenium import webdriver

try:
    URL = "https://GitHub.com/Naereen"
    print(f"Downloading '{URL}'...")
    firefoxOptions = webdriver.FirefoxOptions()
    firefoxOptions.headless = True
    browser = webdriver.Firefox(options=firefoxOptions)

    browser.get('https://GitHub.com/Naereen')
    print(browser.page_source[:500])
finally:
    try:
        browser.close()
    except:
        pass

<html lang="en"><head>
    <meta charset="utf-8">
  <link rel="dns-prefetch" href="https://github.githubassets.com">
  <link rel="dns-prefetch" href="https://avatars0.githubusercontent.com">
  <link rel="dns-prefetch" href="https://avatars1.githubusercontent.com">
  <link rel="dns-prefetch" href="https://avatars2.githubusercontent.com">
  <link rel="dns-prefetch" href="https://avatars3.githubusercontent.com">
  <link rel="dns-prefetch" href="https://github-cloud.s3.amazonaws.com">
  <link rel="d


J'ai bien réussi à installer et utiliser Selenium. Ca marche bien !

## Solution

J'aurai besoin de ces modules là :

In [ ]:
import time
from datetime import datetime
import urllib.request
import subprocess
from selenium import webdriver

Et je vais avoir besoin de cette fonction là, qui vient de https://stackoverflow.com/a/47425305/, pour télécharger l'attestation générée par le site, qui est donnée sous la forme d'une adresse blob:https://media.interieur.gouv.fr/ca78c998-724b-4540-8152-2eb269b06eab

In [39]:
import base64

def get_file_content_chrome(browser, uri):
    """ Use selenium [browser] to download blob [uri].

    - Source https://stackoverflow.com/a/47425305/
    """
    result = browser.execute_async_script("""
        var uri = arguments[0];
        var callback = arguments[1];
        var toBase64 = function(buffer){for(var r,n=new Uint8Array(buffer),t=n.length,a=new Uint8Array(4*Math.ceil(t/3)),i=new Uint8Array(64),o=0,c=0;64>c;++c)i[c]="ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz0123456789+/".charCodeAt(c);for(c=0;t-t%3>c;c+=3,o+=4)r=n[c]<<16|n[c+1]<<8|n[c+2],a[o]=i[r>>18],a[o+1]=i[r>>12&63],a[o+2]=i[r>>6&63],a[o+3]=i[63&r];return t%3===1?(r=n[t-1],a[o]=i[r>>2],a[o+1]=i[r<<4&63],a[o+2]=61,a[o+3]=61):t%3===2&&(r=(n[t-2]<<8)+n[t-1],a[o]=i[r>>10],a[o+1]=i[r>>4&63],a[o+2]=i[r<<2&63],a[o+3]=61),new TextDecoder("ascii").decode(a)};
        var xhr = new XMLHttpRequest();
        xhr.responseType = 'arraybuffer';
        xhr.onload = function(){ callback(toBase64(xhr.response)) };
        xhr.onerror = function(){ callback(xhr.status) };
        xhr.open('GET', uri);
        xhr.send();
    """, uri)
    if type(result) == int :
        raise Exception("Request failed with status %s" % result)
    return base64.b64decode(result)

# bytes = get_file_content_chrome(browser, "blob:https://developer.mozilla.org/7f9557f4-d8c8-4353-9752-5a49e85058f5")

Et maintenant je peux écrire une fonction qui génère une attestation et la sauvegarde, avec les détails (nom, adresse etc) donnés en argument.

Il suffira ensuite de lancer cette fonction toutes les 55 minutes et m'envoyer un mail.

In [67]:
def download_attestation(details):
    """ Fill the form on https://media.interieur.gouv.fr/deplacement-covid-19/ with details, and save the PDF attestation."""
    try:
        firefoxOptions = webdriver.FirefoxOptions()
        firefoxOptions.headless = True
        browser = webdriver.Firefox(options=firefoxOptions)

        URL = "https://media.interieur.gouv.fr/deplacement-covid-19/"
        browser.get(URL)
        page_source = browser.page_source
        page_source_100 = '<html class="fontawesome-i2svg-active fontawesome-i2svg-complete" lang="fr"><head><meta charset="UTF'
        assert page_source[:100] == page_source_100

        # fill with details
        now = datetime.now()
        # fill #field-XXX with YYY
        # - field-firstname
        # - field-lastname
        # - field-birthday
        # - field-placeofbirth
        # - field-address
        # - field-city
        # - field-zipcode
        # - field-datesortie
        # - field-heuresortie
        for fieldname, value in details.items():
            print(f"Filling the form '{fieldname}' with value '{value}'...")
            input_field = browser.find_element_by_id(f"field-{fieldname}")  
            input_field.clear()
            input_field.send_keys(value)
        for fieldname, value in details.items():
            print(f"Checking value of the form '{fieldname}'...")
            input_field = browser.find_element_by_id(f"field-{fieldname}")  
            its_new_value = input_field.get_attribute("value")
            if its_new_value != value:
                print(f"Error: the form '{fieldname}' has value '{its_new_value}' != '{value}'.")

        # click sur '#checkbox-achats'
        print("Clicking on '#checkbox-achats'...")
        checkbox_achats = browser.find_element_by_id("checkbox-achats")
        checkbox_achats.click()

        # click sur '#generate-btn'
        print("Clicking on 'generate-btn'...")
        generate_btn = browser.find_element_by_id("generate-btn")
        # TODO how to configure the path of the file to save?
        generate_btn.click()

        # now wait 5 seconds
        print("New sleeping 5 seconds...")
        time.sleep(5)
        # check that there is a new <a href="..." download="..."> link
        # <a href="blob:https://media.interieur.gouv.fr/712fc9d2-6967-4e63-b9c3-870175b6258f" download="attestation-2020-10-31_12-40.pdf"></a>
        all_a_links = browser.find_elements_by_css_selector("a")
        # window.document.getElementsByTagName("a")
        for link in all_a_links:
            try:
                href = link.get_attribute("href")
                download = link.get_attribute("download")
                if download:
                    print(f"Found a new <a href='...'> link! with href = {href}")
                    print(f"  and it has a download = {download}")
                    print("Downloading the file and save it!")
                    # 1st try...
                    # print("1. using get_file_content_chrome...")
                    bytes_download = get_file_content_chrome(browser, href)
                    download_name = download  #.replace(".pdf", "_2.pdf")
                    with open(download_name, "wb") as download_file:
                        download_file.write(bytes_download)
                    print(f"The PDF file {download_name} is now saved!")
                    ## 2nd try...
                    #print("2. using urllib.request.urlretrieve(href, download)...")
                    #urllib.request.urlretrieve(href, download)
                    #!ls -larth *.pdf
            except:
                pass

    finally:
        try:
            browser.close()
        except:
            pass

In [ ]:
def send_attestations(url_destination):
    

In [43]:
!ls -larth *pdf

-rw-r--r-- 1 lilian lilian 42K oct.  31 13:12 attestation-2020-10-31_13-11_2.pdf
-rw-r--r-- 1 lilian lilian 42K oct.  31 13:13 attestation-2020-10-31_13-13_2.pdf


In [68]:
!CP *.pdf besson@zamok.crans.org:~/www/attestations/

rsync: receiving the argument(s) attestation-2020-10-31_13-11_2.pdf attestation-2020-10-31_13-13_2.pdf besson@zamok.crans.org:~/www/attestations/
Host key fingerprint is SHA256:EJib/9V3VjjkUX6w8bk2zt8BKxmG0JK6D+PDEyLtNb0
+---[ECDSA 256]---+
|     o.       o..|
|    o  .o     o*.|
|     o.+ .   o.++|
|    o ..o .   + +|
|  .  o. S. + . =.|
| . o +o.  o + B +|
|  o ++o... o o * |
|   ..++E.   .   +|
|     .o.        o|
+----[SHA256]-----+
sending incremental file list
created directory /home/besson/www/attestations
attestation-2020-10-31_13-11_2.pdf
         42.87K 100%    4.82MB/s    0:00:00 (xfr#1, to-chk=1/2)
attestation-2020-10-31_13-13_2.pdf
         42.90K 100%    5.84MB/s    0:00:00 (xfr#2, to-chk=0/2)

sent 77.73K bytes  received 106 bytes  51.89K bytes/sec
total size is 85.77K  speedup is 1.10


## Exemple

In [44]:
# TODO read from a file, like a .ini config file
details = {
    'firstname': "Camille",
    'lastname': "Dupont",
    'birthday': "01/01/1970",
    'placeofbirth': "Paris",
    'address': "999 avenue de France",
    'city': "Paris",
    'zipcode': "75001",
    'datesortie': f"{now:%Y-%m-%d}",
    'heuresortie': f"{now:%H:%M}",
}

In [65]:
download_attestation(details)

Filling the form 'firstname' with value 'Camille'...
Filling the form 'lastname' with value 'Dupont'...
Filling the form 'birthday' with value '01/01/1970'...
Filling the form 'placeofbirth' with value 'Paris'...
Filling the form 'address' with value '999 avenue de France'...
Filling the form 'city' with value 'Paris'...
Filling the form 'zipcode' with value '75001'...
Filling the form 'datesortie' with value '2020-10-31'...
Filling the form 'heuresortie' with value '13:13'...
Checking value of the form 'firstname'...
Checking value of the form 'lastname'...
Checking value of the form 'birthday'...
Checking value of the form 'placeofbirth'...
Checking value of the form 'address'...
Checking value of the form 'city'...
Checking value of the form 'zipcode'...
Checking value of the form 'datesortie'...
Checking value of the form 'heuresortie'...
Clicking on '#checkbox-achats'...
Clicking on 'generate-btn'...
New sleeping 5 seconds...


WebDriverException: Message: Failed to decode response from marionette


In [46]:
import subprocess

In [63]:
!ls -larth *.pdf

-rw-r--r-- 1 lilian lilian 42K oct.  31 13:12 attestation-2020-10-31_13-11_2.pdf
-rw-r--r-- 1 lilian lilian 42K oct.  31 13:13 attestation-2020-10-31_13-13_2.pdf


In [64]:
correct = subprocess.check_output(['ls', '-larth', '*.pdf'])
print(correct)

CalledProcessError: Command '['ls', '-larth', '*.pdf']' returned non-zero exit status 2.

With my data:

In [ ]:
# TODO read from a file, like a .ini config file
details = {
    'firstname': "Besson",
    'lastname': "Lilian",
    'birthday': "12/01/1993",
    'placeofbirth': "Briançon",
    'address': "23 boulevard Jeanne d'Arc",
    'city': "Rennes",
    'zipcode': "35000",
    'datesortie': f"{now:%Y-%m-%d}",
    'heuresortie': f"{now:%H:%M}",
}